In [17]:
import pickle
from collections import defaultdict
import re
from dateutil.parser import parse
import spacy

In [3]:
with open("../reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [4]:
# INSPECT_LIST SAMPLE:
# 
# {'abaselinefordetectingmisclassifiedandoutofdistributionexamplesinneuralnetworks': {'AnonReviewer1': {'MEANINGFUL_COMPARISON': [4,
#     datetime.datetime(2016, 12, 19, 0, 0)],
#    'ORIGINALITY': [4, datetime.datetime(2016, 12, 19, 0, 0)],
#    'RECOMMENDATION': [6, datetime.datetime(2016, 12, 19, 0, 0)]},
#   'AnonReviewer2': {'RECOMMENDATION': [6,
#     datetime.datetime(2016, 12, 16, 0, 0)]},
#   'AnonReviewer3': {'IMPACT': [2, datetime.datetime(2017, 1, 20, 0, 0)],
#    'ORIGINALITY': [3, datetime.datetime(2017, 1, 20, 0, 0)],
#    'RECOMMENDATION': [6, datetime.datetime(2017, 1, 20, 0, 0)],
#    'SOUNDNESS_CORRECTNESS': [3, datetime.datetime(2017, 1, 20, 0, 0)]}}

In [5]:
acl_aspects = ["MEANINGFUL_COMPARISON", "IMPACT", "ORIGINALITY", "RECOMMENDATION", "SUBSTANCE", "SOUNDNESS_CORRECTNESS", "APPROPRIATENESS", "CLARITY"]

aspect_dict = defaultdict(dict)
inspect_list = {}

for k in rr:
    new_key = rr[k]["title"].strip().lower()
    
    new_key = re.sub('[^0-9a-zA-Z]+', '', new_key)
    
    aspect_dict[new_key]["title"] = rr[k]["title"]
    aspect_dict[new_key]["dec"] = rr[k]["accepted"]
    aspect_dict[new_key]["pr_id"] = k
    
    inspect_list[new_key] = {}
    
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                if rev["OTHER_KEYS"].find("AnonReviewer") > -1:
                    anon_reviewer = rev["OTHER_KEYS"].split(" ")[-1]
                    if anon_reviewer in inspect_list[new_key]:
                        #inspect_list[new_key][anon_reviewer]
                        dummy = 1
                    else:
                        inspect_list[new_key][anon_reviewer] = {}
                    if "DATE" in rev:
                        if rev["DATE"].find("modified") > -1:
                            d = rev["DATE"]
                            try:
                                rev["DATE"] = re.findall('([0-9]+ [A-Za-z]+ [0-9][0-9][0-9][0-9]\))$', d)[0][:-1]
                            except Exception:
                                print(rev["DATE"])
                        else:
                            try:
                                rev["DATE"] = rev["DATE"].replace("(", "")
                                rev["DATE"] = rev["DATE"].replace(")", "")
                                rev["DATE"] = rev["DATE"].strip()
                                date_of_review = parse(rev["DATE"])
                            except Exception:
                                print(rev["DATE"], k)
                    else:
                        date_of_review = parse("1 Jan 2000")
                    
                    for asp in acl_aspects:
                        if asp in rev:
                            if asp in inspect_list[new_key][anon_reviewer]:
                                prev_date = inspect_list[new_key][anon_reviewer][asp][1]
                                if prev_date < date_of_review:
                                    prev_rev_text = inspect_list[new_key][anon_reviewer][asp][2] + ". " + inspect_list[new_key][anon_reviewer][asp][3]
                                    # current rev text, prev_rev_text
                                    inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"], prev_rev_text]
                                else:
                                    inspect_list[new_key][anon_reviewer][asp][3] = inspect_list[new_key][anon_reviewer][asp][3] + ". " +  rev["TITLE"] + ". " + rev["comments"]
                            else:
                                inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"], " "]
            else:
                for asp in acl_aspects:
                    if asp in rev:
                        inspect_list.append((k, rev))

In [20]:
data_dir = "../../ICLR data/masterdata_unbalanced/"
year = 2017
paper_file = "papers_{}.pkl".format(year)


with open(data_dir+paper_file, "rb") as f:
    papers_data = pickle.load(f)


final_dict = {}

not_found = []

for k in papers_data:
    t = papers_data[k]["content"]["title"].strip().lower()
    t = re.sub('[^0-9a-zA-Z]+', '', t)
    if t in aspect_dict:
        final_dict[k] = aspect_dict[t]
        final_dict[k]["rev_info"] = inspect_list[t]
    else:
#         print(t, k)
        not_found.append(t)
print(len(not_found))

63


In [21]:
nlp = spacy.load("en_core_web_sm")

In [16]:
excel_data = {}
keywords = ["original", "proposal", "originality", "unique", "novelty", "novel", "contribution", "significance", "significant"]

for k final_dict:
    for reviewer_num in final_dict['rev_info']:
        if "ORIGINALITY" in final_dict['rev_info'][reviewer_num]:
            excel_data[k+"_"+reviewer_num] = { "dec": final_dict[k]["dec"],"score": final_dict['rev_info'][reviewer_num]["ORIGINALITY"][0] }
            current_text = final_dict['rev_info'][reviewer_num]["ORIGINALITY"][2]
            other_text = final_dict['rev_info'][reviewer_num]["ORIGINALITY"][3]
            
            doc = nlp(current_text)
            for s in doc.sents:
                sent_text = s.text.lower()
                for kw in 

{'dec': False,
 'pr_id': '618',
 'rev_info': {'AnonReviewer1': {'RECOMMENDATION': [5,
    datetime.datetime(2016, 12, 26, 0, 0),
    'review. This paper presents an improved formulation of CNN, aiming to separate geometric transformation from inherent features. The network can estimate the transformation of filters given the input images. \n\nThis work is based on a solid technical foundation and is motivated by a plausible rationale. Yet, the value of this work in practice is subject to questions:\n\n(1) It relies on the assumption that the input image is subject to a transformation on a certain Lie group (locally). Do such transformations constitute real challenges in practice? State-of-the-art CNNs, e.g. ResNet, are already quite resilient to such local deformations. What such components would add to the state of the art? Limited experiments on Cifar-10 does not seem to provide a very strong argument.\n\n(2) The computational cost is not discussed.',
    ' . review. This paper prese

## Scratch

In [2]:
with open("../features/rev_aspects_2017_peerread.pkl", "rb") as f:
    review_scores = pickle.load(f)

In [3]:
review_scores

{'B1-Hhnslg': {'RECOMMENDATION': [5, 4, 6],
  'dec': False,
  'pr_id': '559',
  'title': 'Prototypical Networks for Few-shot Learning'},
 'B1-q5Pqxl': {'CLARITY': [5, 5],
  'IMPACT': [3],
  'ORIGINALITY': [5],
  'RECOMMENDATION': [7, 6, 6],
  'SUBSTANCE': [5],
  'dec': True,
  'pr_id': '384',
  'title': 'Machine Comprehension Using Match-LSTM and Answer Pointer'},
 'B16Jem9xe': {'RECOMMENDATION': [8, 7],
  'dec': False,
  'pr_id': '534',
  'title': 'Learning in Implicit Generative Models'},
 'B16dGcqlx': {'CLARITY': [5],
  'ORIGINALITY': [1],
  'RECOMMENDATION': [6, 5],
  'SOUNDNESS_CORRECTNESS': [4],
  'dec': True,
  'pr_id': '341',
  'title': 'Third Person Imitation Learning'},
 'B184E5qee': {'CLARITY': [5],
  'IMPACT': [2],
  'ORIGINALITY': [2],
  'RECOMMENDATION': [5, 7, 9],
  'SOUNDNESS_CORRECTNESS': [4],
  'dec': True,
  'pr_id': '339',
  'title': 'Improving Neural Language Models with a Continuous Cache'},
 'B186cP9gx': {'RECOMMENDATION': [3, 4, 4],
  'dec': False,
  'pr_id': '6

In [4]:
review_scores["B16dGcqlx"]

{'CLARITY': [5],
 'ORIGINALITY': [1],
 'RECOMMENDATION': [6, 5],
 'SOUNDNESS_CORRECTNESS': [4],
 'dec': True,
 'pr_id': '341',
 'title': 'Third Person Imitation Learning'}

In [19]:
with open("../features/rev_aspects_2017_peerread.pkl", "rb") as f:
    review_scores = pickle.load(f)

for k in review_scores:
#     print(k, review_scores[k])
#     break
    if k == "B16dGcqlx":
        print(review_scores[k])

{'RECOMMENDATION': [6, 5], 'CLARITY': [5], 'dec': True, 'title': 'Third Person Imitation Learning', 'pr_id': '341', 'SOUNDNESS_CORRECTNESS': [4], 'ORIGINALITY': [1]}


In [6]:
with open("../reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [18]:
rr['341']["reviews"]

[{'IS_META_REVIEW': True,
  'comments': 'The paper extends the imitation learning paradigm to the case where the demonstrator and learner have different points of view. This is an important contribution, with several good applications.  The main insight is to use adversarial training to learn a policy that is robust to this difference in perspective.  This problem formulation is quite novel compared to the standard imitation learning literature (usually first-order perspective), though has close links to the literature on transfer learning (as explained in Sec.2).\n\nThe basic approach is clearly explained, and follows quite readily from recent literature on imitation learning and adversarial training.\n\nI would have expected to see comparison to the following methods added to Figure 3:\n1)  Standard 1st person imitation learning using agent A data, and apply the policy on agent A.  This is an upper-bound on how well you can expect to do, since you have the correct perspective.\n2)  S